#### Runable 协议

In [1]:
from langchain_ollama import OllamaLLM
# from langchain_openai import ChatOpenAI

model = OllamaLLM(model="deepseek-r1:1.5b", base_url="http://localhost:11434")

chunks = []

for chunk in model.stream("天空是什么颜色"):
    chunks.append(chunk)
    print(chunk, end="|", flush=True)
    # print(chunk.content, end="|", flush=True)
    
print("\n",dir(chunk)) # chunk 是一个字符串类型，并不像OpenAI的ChatOpenAI那样有content属性

<think>|
|嗯|，|用户|问|“|天空|是什么|颜色|”。|这个问题|挺|常见的|，|可能|在|广告|、|设计|或者|日常|生活中|经常|遇到|。|首先|，|我要|理解|用户|的需求|。|他们|可能|想|了解|天空|的颜色|如何|影响|整体|视觉|效果|，|或者是|用于|某种|创意|项目|。

|我|应该|考虑|不同|颜色|的|天空|在|不同|文化|中的|表现|，|因为|很多|文化|对|颜色|都有|特别|的看法|和|习惯|。|比如|，在|一些|传统|艺术|作品|中|，|天空|被|描绘|成|紫色|或者|翠|绿|的|，|而|有些人|则|更|喜欢|鲜艳|的|红色|或|橙|色|。

|另外|，|我也|要考虑|用户|是否|知道|天空|的颜色|会影响|整体|的感觉|。|如果|他们|是在|做|广告|，|可能会|强调|使用|柔和|的颜色|以|营造|舒适|的感觉|；|如果是|摄影|，|可能|需要|考虑|色彩|搭配|来|突出|某个|主题|。

|我还|应该|想到|，|天空|的颜色|不仅仅|是一个|简单的|视觉|现象|，|它|也|反映了|文化和|历史|背景|。|不同的|文化|对|天空|的|描绘|和|定义|有|不同|理解|，|这点|在|回答|中|可以|提到|。

|最后|，|用户|可能|想知道|是否有|具体的|科学|解释|，|或者|天空|颜色|的变化|如何|随|时间和|地点|变化|。|考虑到|这些|方面|，|我|可以在|回答|中|简|要|提及|一些|相关|知识|，|而|避免|过于|复杂|。
|</think>|

|天空|的颜色|是一个|广泛|且|复杂|的话题|，|因为它|受|多种|因素|的影响|，|包括|历史|、|文化|、|自然|现象|以及|人类|的|视觉|感知|。|以下|是一|些|常见的|背景|和|色彩|描述|：

|1|.| **|传统|与|宗教|信仰|**|：|许多|地方|的|天空|颜色|与|宗教|或|传统|信仰|有关|：
|  | -| 在|一些|文化|中|，|天空|可能|被|描绘|为|紫色|、|翠|绿|或者|深|红|的颜色|。
|  | -| |佛教|教|义|中有|不同|颜色|的|天|光|，|例如|“|无|畏|”|（|白|）、|“|如|来|”|（|黄|）|等|。

|2|.| **|艺术|与|设计|**|：|在|某些|创意|作品|中|，|天空|可能会|被|描绘|成|柔和

In [2]:
chunks[5],chunks[10],chunks[15]

('问', '”。', '可能')

##### 异步调用

In [3]:
from langchain_ollama import OllamaLLM
import asyncio
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import HumanMessagePromptTemplate

# ChatPromptTemplate.from_messages()期望传入的格式极为严格，
# 每个message必须包含role和content属性，并且content属性必须是一个字符串。
# 如果传入的message不满足这些要求，ChatPromptTemplate.from_messages()会抛出一个ValueError异常。


# 修改这部分，使用HumanMessagePromptTemplate创建符合格式的消息
prompt = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template("给我讲一个关于{topic}的故事")
])

model = OllamaLLM(model="deepseek-r1:1.5b", base_url="http://localhost:11434")
# parser = StrOutputParser()
# chain = prompt | model | parser
chain = prompt | model

async def async_stream():
    async for chunk in chain.stream({"topic":"天空"}):
        print(chunk, end="|", flush=True)

# asyncio.run(async_stream()) # chain.stream({"topic":"天空"}) 返回的是一个普通的生成器对象，
# 而不是一个实现了异步迭代协议（即具有 __aiter__ 和 __anext__ 方法）的异步可迭代对象

# 因此，在调用 asyncio.run() 时，Python 会尝试将生成器对象转换为异步可迭代对象，
# 但由于生成器对象本身并不实现异步迭代协议，因此会引发 TypeError 异常。

In [ ]:
import asyncio
from langchain_ollama import OllamaLLM
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import HumanMessagePromptTemplate


# 包装生成器为异步可迭代对象的类
class AsyncGeneratorWrapper:
    def __init__(self, gen):
        # 初始化时传入一个生成器对象
        self.gen = gen

    def __aiter__(self):
        # 使该类的实例成为异步可迭代对象
        return self

    async def __anext__(self):
        try:
            # 使用 asyncio.to_thread 避免阻塞事件循环，将同步的 next 调用放到单独线程执行
            return await asyncio.to_thread(next, self.gen)
        except StopIteration:
            # 当生成器耗尽时，抛出异步迭代结束的异常
            raise StopAsyncIteration
        except Exception as e:
            # 捕获并打印其他可能出现的异常
            print(f"An error occurred: {e}")
            # 出现异常时也结束异步迭代
            raise StopAsyncIteration


# 创建一个聊天提示模板，接收一个 topic 参数，用于生成用户的提问消息
prompt = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template("给我讲一个关于{topic}的故事")
])

# 初始化 Ollama 语言模型，指定模型名称和服务地址
model = OllamaLLM(model="deepseek-r1:1.5b", base_url="http://localhost:11434")
# 初始化输出解析器，将模型的输出解析为字符串
parser = StrOutputParser()
# 构建一个链式处理流程，先使用提示模板处理输入，再经过模型生成输出，最后用解析器解析输出
chain = prompt | model | parser


async def async_stream():
    # 调用链式处理流程的 stream 方法，传入 topic 参数，得到一个生成器
    gen = chain.stream({"topic": "天空"})
    # 将生成器包装成异步可迭代对象
    async_gen = AsyncGeneratorWrapper(gen)
    # 异步迭代生成器，逐块输出内容
    async for chunk in async_gen:
        # 打印每一块内容，以 | 作为分隔符，并立即刷新输出缓冲区
        print(chunk, end="|", flush=True)


# 根据环境决定是否使用 asyncio.run
try:
    # 获取当前正在运行的事件循环
    loop = asyncio.get_running_loop()
    if loop.is_running():
        # 如果事件循环正在运行，创建一个任务来运行异步函数
        loop.create_task(async_stream())
    else:
        # 如果事件循环未运行，使用 asyncio.run 来运行异步函数
        asyncio.run(async_stream())
except RuntimeError:
    # 如果获取运行的事件循环时出现错误，使用 asyncio.run 来运行异步函数
    asyncio.run(async_stream())

<think>|
|嗯|，|用户|让我|给|一个|关于|天空|的故事|。|首先|，|我|得|想想|什么|故事|适合|天空|？|可能|需要|一个|有趣|且|有|深度|的|设定|。|毕竟|，|天空|是|天|体|的一种|，|有很多|科学|和|艺术|上的|内容|。

|我觉得|古|希腊|是一个|不错|的选择|，|那里|有很多|涉及|天空|的|神话|和|哲学|故事|。|比如|，|赫|拉|克|勒|斯|和|西|西|弗|斯|的故事|，|这些|都|与|永恒|和|循环|有关|。|还有|雅|典|娜|，|她|被|囚|禁|在|海|神| cre|on| 的|牢|笼|中|，|这|涉及到|命运|、|生命|和|永恒|的主题|。

|接下来|，|用户|可能|希望|看到|一个|既有|历史|深度|又|不失|现代|感|的故事|。|所以|，|我|需要|将|古|希腊|神话|中的|元素|融入|到|现代|的|场景|中|去|。|比如|，|把|海|神| cre|on| 换|成|一个|普通人|，|或者|把|故事|的时间|线|调整|为|更|贴近|现实|的生活|。

|再|想想|，|天空|在|故事|中的|角色|可以|如何|发展|。|比如|，|西|西|弗|斯|被|太阳|驱|逐| upwards|，|而不是|被|吸|进去|。|这样|既|有趣|又|符合|古|希腊|神话|的|风格|。|同时|，|海|神| cre|on| 的|命运|也可以|变成|普通|人的|命运|，|增加|故事|的真实|感|和|情感|共鸣|。

|然后|，|我|得|考虑|故事|的|结构|。|开头|可以|介绍|一个|普通人|与|天空|互动|的情|节|，|中间|引|出|西|西|弗|斯|的故事|，|最后|揭示|永恒|的主题|。|这样|结构|清晰|，|逻辑|连|贯|。

|另外|，|用户|可能|还|希望|了解|关于|天空|的一些|科学|知识|，|比如|恒|星|、|地球|自|转|等|，|但|不要|过于|晦|涩|。|可以在|故事|中|自然|地|融入|这些|知识点|，|让|读者|有|更深|的理解|。

|最后|，|结尾|部分|可以|点|出|天空|作为|永恒|存在|的重要性|，|或者|展望|未来|，|为|故事|留下|悬念|或|启发|。|这样|不仅|满足|了|用户|的需求|，|还能|引发|进一步|的|思考|和|讨论|。
|</think>|

|##| 天|空|里的|舞|者|

|人类

In [5]:
import asyncio
# 从 langchain_ollama 库导入 OllamaLLM 类，用于调用 Ollama 模型
from langchain_ollama import OllamaLLM
# 从 langchain_core.output_parsers 库导入 JsonOutputParser 类，用于将模型输出解析为 JSON 格式
from langchain_core.output_parsers import JsonOutputParser

# 初始化 Ollama 语言模型，指定模型名称为 deepseek-r1:1.5b，模型服务的基础 URL 为 http://localhost:11434
model = OllamaLLM(model="deepseek-r1:1.5b", base_url="http://localhost:11434")
# 创建一个链式调用，先使用模型进行推理，然后使用 JsonOutputParser 将输出解析为 JSON 格式
chain = (model | JsonOutputParser())

# 定义一个类，用于将同步生成器包装成异步生成器
class AsyncGeneratorWrapper:
    def __init__(self, gen):
        # 初始化时接收一个同步生成器对象
        self.gen = gen

    def __aiter__(self):
        # 使该类的实例成为异步可迭代对象
        return self

    async def __anext__(self):
        try:
            # 使用 asyncio.to_thread 将同步的 next 调用放到单独的线程中执行，避免阻塞事件循环
            return await asyncio.to_thread(next, self.gen)
        except StopIteration:
            # 当同步生成器耗尽时，抛出异步迭代结束的异常
            raise StopAsyncIteration

# 定义一个异步函数，用于流式处理模型的输出
async def async_stream():
    # 调用 chain 的 stream 方法，传入请求内容，得到一个同步生成器，该请求要求以特定 JSON 格式输出国家及人口列表
    gen = chain.stream(
        "以JSON 格式输出法国、西班牙和日本的国家及人口列表。"
        '使用一个带有“countries”外部键的字典，其中包含国家列表。'
        '每个国家都应该有键“name”和“population”。'
    )
    # 将同步生成器包装成异步生成器
    async_gen = AsyncGeneratorWrapper(gen)
    # 异步迭代异步生成器，逐块获取模型输出
    async for text in async_gen:
        # 打印每一块输出，并立即刷新输出缓冲区
        print(text, flush=True)

try:
    # 获取当前正在运行的事件循环
    loop = asyncio.get_running_loop()
    if loop.is_running():
        # 如果事件循环正在运行，创建一个任务来运行异步函数
        loop.create_task(async_stream())
    else:
        # 如果事件循环未运行，使用 asyncio.run 来运行异步函数
        asyncio.run(async_stream())
except RuntimeError:
    # 如果获取运行的事件循环时出现错误，使用 asyncio.run 来运行异步函数
    asyncio.run(async_stream())

{}
{'countries': {}}
{'countries': {'france': {}}}
{'countries': {'france': {'name': ''}}}
{'countries': {'france': {'name': 'France'}}}
{'countries': {'france': {'name': 'France', 'population': 6}}}
{'countries': {'france': {'name': 'France', 'population': 67}}}
{'countries': {'france': {'name': 'France', 'population': 670}}}
{'countries': {'france': {'name': 'France', 'population': 6700}}}
{'countries': {'france': {'name': 'France', 'population': 67002}}}
{'countries': {'france': {'name': 'France', 'population': 670024}}}
{'countries': {'france': {'name': 'France', 'population': 6700246}}}
{'countries': {}}
{'countries': {'france': {'name': 'France', 'population': 6700246}}}
{'countries': {}}
{'countries': {'france': {'name': 'France', 'population': 6700246}}}
{'countries': {}}
{}
